In [ ]:
pip install peft accelerate bitsandbytes transformers datasets

In [ ]:
!C:/Users/yinch/Desktop/AI_Novelist_Game/venv/Scripts/python.exe -m pip install --upgrade pip

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_from_disk

In [2]:
# 从本地加载 tokenizer 和 dataset
# 加载 tokenizer 和 dataset
model_name = "uer/gpt2-chinese-cluecorpussmall"
tokenizer = AutoTokenizer.from_pretrained("tokenizer_gpt2_chinese/", trust_remote_code=True)
dataset = load_from_disk("data/processed/novel_dataset")


# === 基本配置 ===
dataset_path = "data/processed/novel_dataset"
output_dir = "outputs/gpt2_chinese_lora_small"

In [3]:
# tokenize 数据
def tokenize_function(example):
    return tokenizer(
        example["text"],           # 要编码的字段
        truncation=True,           # 超出 max_length 的内容将被截断
        padding="max_length",      # 不足 max_length 的部分自动填充
        max_length=512             # 每段文本编码成长度为 512 的 token 序列
    )
tokenized_dataset = dataset.map(tokenize_function, batched=True) #告诉 .map() 一次处理多个样本，提高处理速度，example["text"] 在这种情况下是一个列表。

In [4]:
# 加载模型 
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto") 
#加载基础语言模型，例如 DeepSeek 系列。这里不加 device_map="auto" 可能导致超出显存报错，建议加上。

In [5]:
#应用 LoRA
lora_config = LoraConfig(
    r=8,                            # LoRA 的秩，表示适配器的低秩矩阵维度，越大训练越强但显存需求越高
    lora_alpha=16,                 # LoRA 缩放因子，控制训练强度
    target_modules=["c_attn"],  # GPT2结构里较关键的attention层
    lora_dropout=0.05,             # LoRA dropout（防止过拟合）
    bias="none",                   # 不训练 bias 项
    task_type=TaskType.CAUSAL_LM,  # 指定任务类型为因果语言建模（即生成任务）
    fan_in_fan_out=True  # 显式声明，避免警告
)


In [6]:
#将 LoRA 配置注入模型，形成可微调的结构。此时你只会训练少量新增的 LoRA 参数，而不是整个 模型。

model = get_peft_model(model, lora_config)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


In [8]:
# === 训练参数 ===
training_args = TrainingArguments(
    output_dir=output_dir,                  # 训练模型和checkpoint保存的目录
    overwrite_output_dir=True,              # 如果已有同名输出目录，允许覆盖
    num_train_epochs=3,                     # 总共训练3轮（epoch）
    per_device_train_batch_size=1,          # 每块设备（GPU/CPU）上的batch大小设为1，节省显存
    gradient_accumulation_steps=4,          # 梯度累计4步再更新一次，相当于"有效batch size"为4
    save_strategy="steps",                  # ✅ 每隔一定步骤保存
    save_steps=500,                         # ✅ 每 500 步保存一次
    save_total_limit=3,                     # ✅ 最多保留 3 个 checkpoint，自动删旧的
    logging_steps=10,                       # 每10步打印一次训练日志（loss等信息）
    fp16=False,                             # 是否使用混合精度（16-bit float），GTX 1660 不支持，所以设为 False
    logging_dir="logs",                      # logging_dir="logs" 是 TrainingArguments 的一个参数，用来指定 训练日志文件的保存位置，特别是用于生成 TensorBoard 可视化结果 的。
    report_to="none"                        # 不报告日志到W&B等平台
)

In [9]:
from transformers import DataCollatorForLanguageModeling
from transformers import PreTrainedTokenizerBase
# 如果你未来想兼容 5.0，可用 processing_class，等价于 tokenizer（目前不强制）
# 注意：tokenizer 本身无需修改，只是 Trainer 初始化方式可能变化

trainer = Trainer(
    model=model,                      # 你要训练的模型（已注入 LoRA 或其他结构）
    args=training_args,              # 训练配置（刚刚设置的 TrainingArguments）
    train_dataset=tokenized_dataset, # 用于训练的数据集，已经tokenized
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
                                     # 数据收集器，用于动态填充batch，使不同长度的句子可以成批训练
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
print("🚀 开始训练 GPT2 中文小说 LoRA（small）...")
trainer.train()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ 微调完成，模型已保存到:", output_dir)


🚀 开始训练 GPT2 中文小说 LoRA（small）...


C:\Users\yinch\Desktop\AI_Novelist_Game\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.181600
20,4.165400
30,4.192500
40,4.042300
50,3.990200
60,3.839000
70,3.867600
80,3.868600
90,3.822400
100,3.594500


C:\Users\yinch\Desktop\AI_Novelist_Game\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\yinch\Desktop\AI_Novelist_Game\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ 微调完成，模型已保存到: outputs/gpt2_chinese_lora_small
